In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/small_corpus.csv')

In [3]:
df.head()

,Unnamed: 0,ratings,reviews
0,122468,1,Recently UBISOFT had to settle a huge class-ac...
1,495370,1,"code didn't work, got me a refund."
2,214717,1,"these do not work at all, all i get is static ..."
3,231734,1,well let me start by saying that when i first ...
4,154709,1,"Dont waste your money, you will just end up us..."


In [4]:
df.dropna(subset = ["reviews"], inplace=True)

In [5]:
sentences = list(df.reviews.values)

In [7]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
inputs = tokenizer(sentences, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [12]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [13]:
rand = torch.rand(inputs.input_ids.shape)

In [14]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [15]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

<ipython-input-15-d8b90b293d85>:5: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:983.)
  torch.flatten(mask_arr[i].nonzero()).tolist()


In [16]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [17]:
inputs.attention_mask

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [18]:
from torch.utils.data import TensorDataset,random_split
dataset = TensorDataset(inputs.input_ids,inputs.attention_mask,inputs.labels)

In [19]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

In [20]:
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [22]:
from transformers import AdamW, BertConfig


# Tell pytorch to run this model on the GPU.
model.cuda()
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [23]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [24]:
import numpy as np
import random
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [25]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
 for input_ids,mask,labels in train_dataloader:
        input_ids,mask,labels = input_ids.to(device), mask.to(device),labels.to(device)
        out = model(input_ids, token_type_ids=None, attention_mask=mask, labels=labels,return_dict=True)
        print(out)
        break;

NameError: name 'train_dataloader' is not defined

In [32]:

torch.cuda.empty_cache()

NameError: name 'labels' is not defined

In [26]:
from fastprogress.fastprogress import master_bar, progress_bar
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def fit(epochs,model,train_dl,valid_dl,opt):
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','trn_acc','val_acc'],table=True)

    for i in mb:    
        trn_loss,val_loss = 0.0,0.0
        trn_acc,val_acc = 0,0
        trn_n,val_n = len(train_dl.dataset),len(valid_dl.dataset)
        model.train()
        for input_ids,mask,labels in progress_bar(train_dl,parent=mb):
            input_ids,mask,labels = input_ids.to(device), mask.to(device),labels.to(device)
            out = model(input_ids, 
                       token_type_ids=None, 
                       attention_mask=mask, 
                       labels=labels,
                       return_dict=True)
            opt.zero_grad()
            loss = out[0]
            trn_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            scheduler.step()
        trn_loss /= mb.child.total
        trn_acc /= trn_n

        model.eval()
        with torch.no_grad():
            for input_ids,mask,labels in progress_bar(valid_dl,parent=mb):
                input_ids,mask,labels = input_ids.to(device), mask.to(device),labels.to(device)
                result = model(input_ids, 
                           token_type_ids=None, 
                           attention_mask=mask,
                           labels=labels,
                           return_dict=True)
                loss = result.loss
                logits = result.logits
                val_loss += loss.item()
        val_loss /= mb.child.total
        val_acc /= val_n

        mb.write([i,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{trn_acc:.6f}',f'{val_acc:.6f}'],table=True)

In [ ]:
fit(2,model=model,train_dl=train_dataloader,valid_dl=validation_dataloader,opt=optimizer)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,1.552376,0.112873,0.000000,0.000000


In [28]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [30]:
dump_tensors()

/opt/conda/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:144: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


Parameter: GPU pinned 30522 × 768
Parameter: GPU pinned 512 × 768
Parameter: GPU pinned 2 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 30522
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 3072 × 768
Parameter: GPU pinned 3072
Parameter: GPU pinned 768 × 3072
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 3072 × 768
Parameter: GPU pinned 3072
Parameter: GPU pinned 768 × 3072
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768 × 76